<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h1>
    Auxiliar Files to PostgresSQL
    </h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h3>
        Documentation
        <br>
        ____________________
            </br>
    </h3>
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div id="outilne">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #292929; padding: 10px; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #E0E0E0;">
        Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #94FFD8; font-size: 18px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #94FFD8; font-size: 18px;">
        Initial set-up</a>
    <br>
    <a href="#1" style="color: #94FFD8; font-size: 18px;">
        1. Loading gdp_percentage_structure</a>
    <br>
    <a href="#2" style="color: #94FFD8; font-size: 18px;">
        2. Generate PDF input with key tables</a>
    <br>
    <a href="#3" style="color: #94FFD8; font-size: 18px;">
        3. Load dummies for raw data delivered by Central Bank</a>
</div>


<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123)">email Jason Cruz
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px"">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following code lines will create folders in your current path, call them to import and export your outputs. <div/>

In [2]:
# Folder path to save the csv file of GDP percentage structure

gdp_perc_structure_folder = 'gdp_perc_structure'
if not os.path.exists(gdp_perc_structure_folder):
    os.makedirs(gdp_perc_structure_folder) # to create the folder (if it doesn't exist)

In [3]:
# Folder path to save the csv file of NS dates

ns_dates_folder = 'ns_dates'
if not os.path.exists(ns_dates_folder):
    os.makedirs(ns_dates_folder) # to create the folder (if it doesn't exist)

In [4]:
# Folder path to save the xlsx files delivered by Central Bank

data_delivered_folder = 'old_raw_data_delivered'
if not os.path.exists(data_delivered_folder):
    os.makedirs(data_delivered_folder) # to create the folder (if it doesn't exist)

<p style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following function will establish a connection to the <code>gdp_revisions_datasets</code> database in <code>PostgreSQL</code>. The <b>input data</b> used in this jupyter notebook will be loaded from this <code>PostgreSQL</code> database, and similarly, all <b>output data</b> generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.<p/>
    
<p style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
To request permissions, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"> <span style="font-size: 24px;">&#x2709;</span>
    </a>.
<p/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Enter your user credentials to acces to SQL.
    <div/>

In [44]:
# Get environment variables
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Check if all environment variables are defined
if not all([host, user, password]):
    raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Create connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Import all other functions required by this jupyter notebook.
    </span>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Please, check the script <code>gdp_revisions_datasets_functions.py</code> which contains all the functions required by this jupyter notebook. The functions there are ordered according to the <a href="#outilne" style="color: #3d30a2;">sections</a> of this jupyter notebok.<div/>

In [45]:
from gdp_revisions_datasets_functions import *

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading gdp_percentage_structure</span></h1>
    </div>

In [ ]:
# Loading csv data
gdp_percentage_structure = f'{gdp_perc_structure_folder}/gdp_percentage_structure.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
gdp_percentage_structure = pd.read_csv(gdp_percentage_structure, delimiter=';')
gdp_percentage_structure.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
gdp_percentage_structure.to_sql('gdp_percentage_structure', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Load date, year and id data</span></h1>
    </div>

In [ ]:
# Loading csv data
csv_file = f'{ns_dates_folder}/dates.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
dates = pd.read_csv(csv_file, delimiter=';')
dates.head(5)

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Clean-up the dataframe columns.<div/>

In [ ]:
# Convert the 'date' column to datetime type without time
dates['date'] = pd.to_datetime(dates['date'], format='%d/%m/%Y', errors='coerce')

# Check the data type of the 'date' column after conversion
print(dates['date'].dtype)

# Check if there are NaT values in the 'date' column
print(dates['date'].isnull().sum())

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Loading to SQL.<div/>

In [ ]:
dates.to_sql('dates_growth_rates', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Load dummies for raw data delivered by Central Bank</span></h1>
    </div>

In [6]:
# Loading csv data
delivered_data_dummies = f'{data_delivered_folder}/old_raw_data_delivered.csv'

# Reads the CSV file in a DataFrame, specifying that the delimiter is the comma
delivered_data_dummies = pd.read_csv(delivered_data_dummies, delimiter=';')
delivered_data_dummies.head(5)

,year,id_ns,delivered_1,delivered_2
0,1994,1,0,0
1,1994,2,0,0
2,1994,3,1,0
3,1994,4,0,0
4,1994,5,0,0


In [7]:
delivered_data_dummies.to_sql('old_raw_data_delivered', engine, index=False, if_exists='replace')

939

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div style="background-color: #00414C; color: white; padding: 10px;">
<h1><span style = "color: #15F5BA; font-family: 'PT Serif Pro Book'; color: dark;">4.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Concatenate old_gdp_dataset with new_gdp_dataset</span></h1>
    </div>

<div id="steps-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-4-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Select economic sector</a>
    <br>
    <a href="#step-4-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Select frequency</a>
    <br>
    <a href="#step-4-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Import dataframes from PostgresSQL</a>
    <br>
    <a href="#step-4-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Concatenate dataframes</a>
    <br>
    <a href="#step-4-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Load to PostgresSQL</a>
</div>

<div id="step-4-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Select economic sector</span>
  </div>

In [ ]:
# Call the function to show the popup window
sector = show_option_window()
print("Selected economic sector:", sector)

<div id="step-4-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Select frequency</span>
  </div>

In [47]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

Selected frequency: monthly


<div id="step-4-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Import dataframes from PostgresSQL</span>
  </div>

In [38]:
# Crear un DataFrame de pandas desde la tabla
query_1 = f"SELECT * FROM old_{sector}_{frequency}_growth_rates;"
df_1 = pd.read_sql(query_1, engine)

   year id_ns       date  oct_2010  nov_2010
0  2012    01 2012-01-06       8.5      10.2
1  2012    02 2012-01-13       8.5      10.2
2  2012    03 2012-01-20       NaN      10.2


In [39]:
# Crear un DataFrame de pandas desde la tabla
query_2 = f"SELECT * FROM new_{sector}_{frequency}_growth_rates;"
df_2 = pd.read_sql(query_2, engine)

   oct_2022  nov_2022
0       2.3       2.1
1       2.3       2.1
2       NaN       2.1


<div id="step-4-4">
   <!-- Contenido de la celda de destino -->
</div>

<div id="step-4-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> <span>Concatenate dataframes</span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Comment the code below if you want the default option (display rows and columns of the dataframe in a limited way)
    <div/>

In [41]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Function to create concatenated dataframe
    <div/>

In [42]:
def concatenate_dataframes(df1, df2):
    # Concatenate the dataframes
    concatenated_df = pd.concat([df1, df2], ignore_index=True)
    
    return concatenated_df

# Call the function
globals()[f'{sector}_{frequency}_growth_rates'] = concatenate_dataframes(df_1, df_2)
df = globals()[f'{sector}_{frequency}_growth_rates']

# Display the resulting dataframe
df.head(100)

,year,id_ns,date,oct_2010,nov_2010,dic_2010,ene_2011,feb_2011,mar_2011,abr_2011,may_2011,jun_2011,jul_2011,ago_2011,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023,ene_2024,feb_2024,mar_2024,abr_2024
0,2012,01,2012-01-06,8.5,10.2,8.9,10.2,8.5,8.1,7.4,7.5,5.6,6.3,7.5,5.8,5.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,02,2012-01-13,8.5,10.2,8.9,10.2,8.5,8.1,7.4,7.5,5.6,6.3,7.5,5.8,5.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,03,2012-01-20,NaN,10.2,8.9,10.2,8.5,8.1,7.6,7.4,5.5,6.4,7.6,5.8,5.1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,04,2012-01-27,NaN,10.2,8.9,10.2,8.5,8.1,7.6,7.4,5.5,6.4,7.6,5.8,5.1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012,05,2012-02-03,NaN,10.2,8.9,10.2,8.5,8.1,7.6,7.4,5.5,6.4,7.6,5.8,5.1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2012,06,2012-02-10,NaN,10.2,8.9,10.2,8.5,8.1,7.6,7.4,5.5,6.4,7.6,5.8,5.1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2012,07,2012-02-17,NaN,10.2,8.9,10.2,8.5,8.1,7.6,7.4,5.5,6.4,7.6,5.8,5.1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2012,08,2012-02-24,NaN,NaN,8.9,10.2,8.3,8.0,7.8,7.4,5.4,6.5,7.7,5.9,5.3,5.1,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2012,09,2012-03-02,NaN,NaN,8.9,10.2,8.3,8.0,7.8,7.4,5.4,6.5,7.7,5.9,5.3,5.1,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2012,10,2012-03-09,NaN,NaN,8.9,10.2,8.3,8.0,7.8,7.4,5.4,6.5,7.7,5.9,5.3,5.1,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div id="step-4-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> <span>Load to PostgresSQL</span>
  </div>

In [ ]:
#df.to_sql(f'{sector}_{frequency}_growth_rates', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-4" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-4" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-4-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-4-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>